# EDA - Customer chrun Dataset 
Goal is to understand data patterns, identify features, clean the dataset and build the baselin

# Step 1 is to load the dataset 

In [85]:
import pandas as pd 

df = pd.read_csv("../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head() #shows first 5 rows of the dataframe

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


# step two is to do a basic exploration of the dataset 
Are columns numeric or categorical?
Any missing values?
Is the target balanced?

lesson learnt: mechanistically try and identify what you want to know (e.g which columns do you want to be numerical, are there any acceptable null values or only a certain column and then systematically go through what functions will apply to that criteria) 

In [86]:
df.info() #shiows non-null counts
df.describe(include="all") #looks at numerical and non-numerical columns
df._get_numeric_data() #shows only numerical columns
df.isnull #shows null values
df['Churn'].value_counts() #shows counts of unique values in 'Churn' column
df.nunique() #shows number of unique values in each column
#(data.isnull().sum()/(len(data)))*100 #percentage of missing values in each column



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64

# Minimal clean up

In [87]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce') #convert TotalCharges to numeric, coerce errors to NaN
print(df["TotalCharges"].dtype) #check datatype
if 'customerID' in df.columns:
    del df['customerID'] #remove customerID column if exists
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0}) #map Yes/No to 1/0 in Churn column
df['Churn'].unique()
#data.drop_duplicates(inplace=True) #remove duplicates if any
#can also use df.duplicated().sum() to check number of duplicates`
# correct inconsistent data formats = e.g.  data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')`

float64


array([0, 1])

In [88]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


# Train/Test Split 

In [89]:
from sklearn.model_selection import train_test_split

X = df.drop('Churn', axis=1)
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

y_test.mean()
y_train.mean()

np.float64(0.2653532126375577)

# Encode and Split
encode after train/testing due to data leakage risks, ,odel genrealisation, consistenecy and practicality 

In [90]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# 1. identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns #identify categorial columns
print(categorical_cols)

# 2. create ColumnTransformer for OHE
preprocessor = ColumnTransformer(
    transformers=[
        ( 'categorical preprocessing', 
         OneHotEncoder(handle_unknown='ignore'), #handle_unknown = 'ignore' to avoid errors during prediction
            categorical_cols
        ) 
    ],
    remainder='passthrough'  # keep other columns unchanged
)
model = RandomForestClassifier(n_estimators=100, random_state=42)
# create a Pipeline with preprocessing + classifier
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

# fit the model on training data
clf.fit(X_train, y_train)

#print shape 
print(X_train.shape, X_test.shape)
print(y_train.mean(), y_test.mean())  # check churn rate is similar




Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object')
(5634, 19) (1409, 19)
0.2653532126375577 0.2654364797728886


# Evaluation of model 

In [91]:
from sklearn.metrics import accuracy_score, roc_auc_score

# get predictions (preds) and probabilities (probs)
preds = clf.predict(X_test)
probs = clf.predict_proba(X_test)[:, 1]

# compute accuracy
accuracy = accuracy_score(y_test, preds) #why y_test and preds? 
# compute ROC-AUC
auc = roc_auc_score(y_test, probs)
print( "Accuracy:", accuracy
      , "ROC-AUC:", auc )


Accuracy: 0.7842441447835344 ROC-AUC: 0.8189116226200625


# Manual Test

In [95]:
clf.predict(X_test.iloc[:5])


array([0, 1, 0, 0, 0])

# save trained model and check it 

In [96]:
import joblib 
import os

filename = "../src/train/baseline_model.pkl"
os.makedirs("../src/train/", exist_ok=True)
joblib.dump(clf, filename)

['../src/train/baseline_model.pkl']

In [97]:
from sklearn.metrics import classification_report
# load model with joblib
loaded_model = joblib.load("../src/train/baseline_model.pkl")


# check results
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1035
           1       0.62      0.48      0.54       374

    accuracy                           0.78      1409
   macro avg       0.72      0.69      0.70      1409
weighted avg       0.77      0.78      0.78      1409

